# CAT-Loss War Room

> **DEMO RESEARCH MEMO** — This notebook is for demonstration purposes only.
> It is **not legal advice**. All outputs must be independently verified.
> **Verify all citations** (KeyCite / Shepardize) before any legal reliance.

---

AI-powered catastrophic loss litigation research. Given a case intake, this notebook
generates targeted research queries across weather data, carrier intel, and case law.

In [1]:
"""Cell 1: Imports + Configuration"""
import sys, os
from pathlib import Path

# Ensure src/ is on the path
ROOT = Path(".").resolve().parent
if str(ROOT / "src") not in sys.path:
    sys.path.insert(0, str(ROOT / "src"))

from dotenv import load_dotenv
load_dotenv(ROOT / ".env")

# Config
USE_CACHE = os.getenv("USE_CACHE", "true").lower() == "true"
CACHE_DIR = str(ROOT / os.getenv("CACHE_DIR", "cache"))
CACHE_SAMPLES_DIR = str(ROOT / os.getenv("CACHE_SAMPLES_DIR", "cache_samples"))
OUTPUT_DIR = str(ROOT / os.getenv("OUTPUT_DIR", "output"))

print("=" * 50)
print("WAR ROOM CONFIG")
print("=" * 50)
print(f"  USE_CACHE:          {USE_CACHE}")
print(f"  CACHE_DIR:          {CACHE_DIR}")
print(f"  CACHE_SAMPLES_DIR:  {CACHE_SAMPLES_DIR}")
print(f"  OUTPUT_DIR:         {OUTPUT_DIR}")
print(f"  EXA_API_KEY:        {'***set***' if os.getenv('EXA_API_KEY') else 'NOT SET'}")
print("=" * 50)

WAR ROOM CONFIG
  USE_CACHE:          True
  CACHE_DIR:          C:\Users\user\cat-loss-war-room-demo\cache
  CACHE_SAMPLES_DIR:  C:\Users\user\cat-loss-war-room-demo\cache_samples
  OUTPUT_DIR:         C:\Users\user\cat-loss-war-room-demo\output
  EXA_API_KEY:        ***set***


In [2]:
"""Cell 2: Case Intake"""
from war_room.query_plan import CaseIntake

intake = CaseIntake(
    event_name="Hurricane Milton",
    event_date="2024-10-09",
    state="FL",
    county="Pinellas",
    carrier="Citizens Property Insurance",
    policy_type="HO-3 Dwelling",
    posture=["denial", "bad_faith"],
    key_facts=[
        "Category 3 at landfall near Siesta Key",
        "Roof damage + water intrusion reported within 48 hours",
        "Claim denied citing pre-existing conditions",
    ],
    coverage_issues=[
        "wind vs water causation",
        "anti-concurrent causation clause",
        "duty to investigate",
    ],
)

print(intake.format_card())

CASE INTAKE
  Event:       Hurricane Milton (2024-10-09)
  Location:    Pinellas County, FL
  Carrier:     Citizens Property Insurance
  Policy:      HO-3 Dwelling
  Posture:     denial, bad_faith
  Key Facts:   Category 3 at landfall near Siesta Key; Roof damage + water intrusion reported within 48 hours; Claim denied citing pre-existing conditions
  Issues:      wind vs water causation; anti-concurrent causation clause; duty to investigate


In [3]:
"""Cell 3: Query Plan Generation"""
from war_room.query_plan import generate_query_plan, format_query_plan

queries = generate_query_plan(intake)
print(format_query_plan(queries))

QUERY PLAN — 18 queries

  [WEATHER DATA] (5 queries)
  ----------------------------------------
  [damage_report] Hurricane Milton Pinellas County FL damage report [from 2024-10-09] (prefer: noaa.gov, weather.gov)
  [wind_data] NWS Hurricane Milton wind speed Pinellas FL [from 2024-10-09] (prefer: weather.gov, nhc.noaa.gov)
  [flood_surge] Hurricane Milton storm surge flood Pinellas County FL [from 2024-10-09] (prefer: noaa.gov, usgs.gov)
  [fema_declaration] FEMA disaster declaration Hurricane Milton FL [from 2024-10-09] (prefer: fema.gov, disasterassistance.gov)
  [loss_estimate] Hurricane Milton damage assessment Pinellas County insured losses [from 2024-10-09]

  [CARRIER DOCUMENTS] (5 queries)
  ----------------------------------------
  [denial_patterns] Citizens Property Insurance Hurricane Milton claim denial FL [from 2024-10-09]
  [doi_complaints] Citizens Property Insurance department of insurance complaints FL (prefer: floir.com, tdi.texas.gov, naic.org)
  [regulatory_actio

In [4]:
"""Cell 4: Weather Corroboration"""
from war_room.exa_client import ExaClient
from war_room.weather_module import build_weather_brief

# Initialize Exa client (only needed if USE_CACHE=False)
client = None
try:
    client = ExaClient()
except (ValueError, Exception):
    print("No EXA_API_KEY set — running from cache only")

weather = build_weather_brief(
    intake, client,
    use_cache=USE_CACHE,
    cache_dir=CACHE_DIR,
    cache_samples_dir=CACHE_SAMPLES_DIR,
)

print(f"Event: {weather['event_summary']}")
print(f"Sources: {len(weather['sources'])}")
print(f"Metrics: {weather['metrics']}")
print()
print("Key Observations:")
for i, obs in enumerate(weather['key_observations'][:5], 1):
    print(f"  {i}. {obs[:150]}")
print()
print("Top Sources:")
for s in weather['sources'][:5]:
    print(f"  {s['badge']} {s['title'][:60]}")
    print(f"    {s['url']}")

Event: Hurricane Milton — Pinellas County, FL (2024-10-09)
Sources: 15
Metrics: {'max_wind_mph': 180, 'storm_surge_ft': 10.0, 'rain_in': None}

Key Observations:
  1. NATIONAL HURRICANE CENTER
TROPICAL CYCLONE REPORT
HURRICANE MILTON
(AL142024)
5–10 October 2024
John L. Beven II, Laura Alaka, and Cody Fritz
National
  2. Storm Events Database - Event Details | National Centers for Environmental Information
[National Centers for Environmental Information] 
[] 
* [Home] 
  3. POST TROPICAL CYCLONE REPORT
Storm Name Hurricane Milton
NWS Office Tampa Bay Area/Ruskin
Begin/End Date 10/08/2024-10/10/2024
Fatalities 3 - Direct
7
  4. Storm Events Database - Event Details | National Centers for Environmental Information
[National Centers for Environmental Information] 
[] 
* [Home] 
  5. Department of Commerce // National Oceanic and Atmospheric Administration // 1
National 
Weather 
Service 
Lessons from the 2024 Hurricane 
Season and

Top Sources:
  🟢 [PDF] Hurricane Milton
    https://www.n

In [5]:
"""Cell 5: Carrier Document Pack"""
from war_room.carrier_module import build_carrier_doc_pack

carrier = build_carrier_doc_pack(
    intake, client,
    use_cache=USE_CACHE,
    cache_dir=CACHE_DIR,
    cache_samples_dir=CACHE_SAMPLES_DIR,
)

print(f"Carrier: {carrier['carrier_snapshot']['name']}")
print(f"Documents: {len(carrier['document_pack'])}")
print()
print("Document Pack:")
for d in carrier['document_pack'][:8]:
    print(f"  {d['badge']} [{d['doc_type']}] {d['title'][:60]}")
print()
print("Common Defenses:")
for d in carrier['common_defenses']:
    print(f"  - {d}")
print()
print("Rebuttal Angles:")
for r in carrier['rebuttal_angles']:
    print(f"  -> {r}")

Carrier: Citizens Property Insurance
Documents: 12

Document Pack:
  🔴 [Denial Pattern Analysis] Citizens sued over alleged mishandling of Hurricane Milton c
  🟢 [Denial Pattern Analysis] Commissioner Yaworsky Fights for Consumers and Brings More .
  🔴 [Denial Pattern Analysis] Denied By Citizens Property Insurance For Your Hurricane Deb
  🔴 [Denial Pattern Analysis] Denied Hurricane Milton Insurance Claim? Here's What to Do |
  🔴 [Denial Pattern Analysis] The Reasons Behind Citizens Claim 'Denials' - Lisa Miller As
  🟢 [DOI/Regulatory Complaint] Consumers - floir
  🟢 [DOI/Regulatory Complaint] Organization and Operation - floir
  🟢 [DOI/Regulatory Complaint] Contact Us - floir

Common Defenses:
  - Citizens Property Insurance may argue damage was pre-existing
  - Policy exclusion defense

Rebuttal Angles:
  -> Key fact undermines carrier position: Category 3 at landfall near Siesta Key
  -> Key fact undermines carrier position: Roof damage + water intrusion reported within 48 hours
  

In [6]:
"""Cell 6: Case Law + Citation Spot-Check"""
from war_room.caselaw_module import build_caselaw_pack
from war_room.citation_verify import spot_check_citations

caselaw = build_caselaw_pack(
    intake, client,
    use_cache=USE_CACHE,
    cache_dir=CACHE_DIR,
    cache_samples_dir=CACHE_SAMPLES_DIR,
)

print("Case Law by Issue:")
for issue in caselaw['issues']:
    print(f"\n  [{issue['issue']}]")
    for c in issue['cases']:
        cite = f" -- {c['citation']}" if c.get('citation') else ''
        print(f"    {c['badge']} {c['name'][:50]}{cite}")

print("\n" + "=" * 50)
print("Citation Spot-Check")
print("=" * 50)

citecheck = spot_check_citations(
    caselaw, client,
    use_cache=USE_CACHE,
    cache_dir=CACHE_DIR,
    cache_samples_dir=CACHE_SAMPLES_DIR,
)

for c in citecheck['checks']:
    print(f"  {c['badge']} {c['case_name'][:40]} -- {c['note'][:60]}")

s = citecheck['summary']
print(f"\nSummary: {s['verified']} verified, {s['uncertain']} uncertain, {s['not_found']} not found")
print(f"\n{citecheck['disclaimer']}")

Case Law by Issue:

  [Citizens Property Insurance Precedent]
    🟡 Sapuppo v. Allstate Floridian Ins. Co., 739 F.3d 6 -- 739 F.3d 678
    🟡 Siegle v. Progressive Consumers Ins. Co., 819 So.  -- 819 So. 2d 732

  [Coverage / Denial Law]
    🟡 Quesada v. Director, Fed. Emergency Agency, 753 F. -- 753 F.2d 1011
    🟡 Quesada v. Director, Federal Emergency Mgmt. Agenc -- 577 F. Supp. 695
    🟡 Johnson v. Davis, 480 So. 2d 625 -- 480 So. 2d 625

  [Concurrent Causation Doctrine]
    🔴 Hurricane Irma – The State of Concurrent Causation -- 208 So. 3d 694
    🔴 Wind v. Flood in the Wake of Hurricane Ian - Butle

  [Coverage Issue]
    🔴 Wind vs. Water Damage: What Homeowners Must Know

Citation Spot-Check
  ⚠️ Sapuppo v. Allstate Floridian Ins. Co.,  -- Found on professional source: casetext.com — verify independ
  ✅ Siegle v. Progressive Consumers Ins. Co. -- Found on official source: supremecourt.flcourts.gov
  ⚠️ Quesada v. Director, Fed. Emergency Agen -- Found on professional source: cas

In [7]:
"""Cell 7: Export Research Memo"""
from war_room.export_md import render_markdown_memo, write_markdown

memo_md = render_markdown_memo(
    intake, weather, carrier, caselaw, citecheck, queries
)

case_key = "milton_citizens_pinellas"
output_path = write_markdown(OUTPUT_DIR, case_key, memo_md)

print(f"Memo saved to: {output_path}")
print(f"Length: {len(memo_md)} characters")
print()
# Show first 40 lines as preview
for line in memo_md.split('\n')[:40]:
    print(line)

Memo saved to: C:\Users\user\cat-loss-war-room-demo\output\milton_citizens_pinellas_20260225_104046.md
Length: 21513 characters

# CAT-Loss War Room — Research Memo

> **DRAFT — ATTORNEY WORK PRODUCT**
>
> **DEMO RESEARCH MEMO — VERIFY CITATIONS — NOT LEGAL ADVICE**
>
> Generated: 2026-02-25 10:40

---

## Case Intake

- **Event:** Hurricane Milton (2024-10-09)
- **Location:** Pinellas County, FL
- **Carrier:** Citizens Property Insurance
- **Policy:** HO-3 Dwelling
- **Posture:** denial, bad_faith
- **Key Facts:** Category 3 at landfall near Siesta Key; Roof damage + water intrusion reported within 48 hours; Claim denied citing pre-existing conditions
- **Coverage Issues:** wind vs water causation; anti-concurrent causation clause; duty to investigate

---

## Weather Corroboration

**Hurricane Milton — Pinellas County, FL (2024-10-09)**

### Metrics Extracted

- Max Wind: **180 mph**
- Storm Surge: **10.0 ft**

### Key Observations

- NATIONAL HURRICANE CENTER
TROPICAL CYCLONE REPORT